In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random 
import math
#Load Pre-trained Model
module_handle =hub.load("https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1").signatures['default']
#Define Color Dictionary
colorcodes={}
#Ye function ek bounding box draw karta hai
def drawbox(image,ymin,xmin,ymax,xmax,namewithscore,color):
    #Isme image ki height aur width ke hisaab se bounding box ke coordinates
    im_height,im_with,_ =image.shape
    left,top,right,bottom=int(xmin*im_with),int(ymin*im_height),int(xmax*im_with),int(ymax*im_height)
    #Ye line bounding box draw karti hai specified coordinates par aur color ke saath.
    cv2.rectangle(image,(left,top),(right,bottom),color = color,thickness=1)
    #Yahan text aur bounding box ki thickness aur scale ko calculate kar rahe hain based on bounding box size.
    FONT_SCALE=5e-3
    THICKNESS_SCALE=4e-3
    width=right-left
    height=bottom-top
    TEXT_Y_OFFSET_SCALE=1e-2
    #Draw Filled Rectangle for Text Background
    cv2.rectangle(image,(left,top-int(height*6e-2)),(right,top),color=color,thickness=-1)
    #Add Text (Object Name and Score)
    cv2.putText(image,namewithscore,
                (left,top-int(height*TEXT_Y_OFFSET_SCALE)),
                fontFace=cv2.FONT_HERSHEY_PLAIN,
                fontScale=min(width,height)*FONT_SCALE,
                thickness= math.ceil(min(width, height)*THICKNESS_SCALE), 
                color=(255,255,255))
def drow(image,boxes,classnames,scores):
    boxesidx=tf.image.non_max_suppression(boxes,scores,max_output_size=20,score_threshold=0.2)
    for i in boxesidx:
        ymin,xmin,ymax,xmax=tuple(boxes[i])
        classname =classnames[i].decode("ascii")
        if classname in colorcodes.keys():
            color=colorcodes[classname]
        else:
            c1=random.randrange(0,255,30)
            c2=random.randrange(0,255,25)
            c3=random.randrange(0,255,50)
            colorcodes.update({classname:(c1,c2,c3)})
            color=colorcodes[classname]
        namewithscore=" {}:{}".format(classname,int(100*scores[i]))
        drawbox(image,ymin,xmin,ymax,xmax,namewithscore,color)
    
    return image
image = cv2.imread("traf.jpg")
image2= cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
converted_img=tf.image.convert_image_dtype(image2,tf.float32)[tf.newaxis,...]

detection=module_handle(converted_img)

result={key:value.numpy() for key,value in detection.items()}
imagewithboxes=drow(image,result["detection_boxes"],result['detection_class_entities'],result['detection_scores'])

cv2.imwrite("traf1.jpg",imagewithboxes)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


True